
# Modelamiento e identificación del UNThermal

El UNThermal es un minilaboratorio que permite realizar el control de temperatura de un par transistor-sensor mediante la administración precisa de potencia eléctrica controlada por una fuente de corriente. 


## 1. Configuración


### Detectando el broker 

Este código sirve para detectar la dirección del broker que está en el laboratorio. Descomente solamente para saber donde está el broker del laboratorio. __Asegúrese de instalar [nmap](https://nmap.org/download.html#windows/ "nmap").__

In [60]:
# s = !nmap -sn 10.203.162.*  
# macindex = [index for index, string in enumerate(s) if  'MAC Address: B8:27:EB:EB:A9:10' in string]
# b = s[macindex[0]-2]
# broker = b.split()[-1]
# print(f"please connect to broker {broker:s}")


### Instalación de la libreria unthermal

Descomentar y ejecutar esta celda solo para instalar por primera vez o actualizar la libreria. __Asegúrese de instalar  [Git](https://git-scm.com/download/win/ "Git").__


In [8]:
import subprocess
command = ["pip", "install", "-I", "git+https://github.com/nebisman/UNThermal.git@main#subdirectory=code/python_code"]
process = subprocess.Popen(command, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)
for line in process.stdout:
    print(line.strip())

Cloning https://github.com/nebisman/UNThermal.git (to revision main) to /tmp/pip-req-build-3c9fwbiy
Running command git clone --filter=blob:none --quiet https://github.com/nebisman/UNThermal.git /tmp/pip-req-build-3c9fwbiy
Resolved https://github.com/nebisman/UNThermal.git to commit add8d77265ce2f4d57f9dba551ac3a502f56e9c2
Installing build dependencies: started
Installing build dependencies: finished with status 'done'
Getting requirements to build wheel: started
Getting requirements to build wheel: finished with status 'done'
Preparing metadata (pyproject.toml): started
Preparing metadata (pyproject.toml): finished with status 'done'
Using cached iottalk_paho_mqtt-1.5.1-py3-none-any.whl.metadata (46 kB)
Using cached control-0.10.0-py3-none-any.whl.metadata (7.6 kB)
Using cached numpy-2.0.1-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (60 kB)
Using cached matplotlib-3.9.2-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (11 kB)
Using cached sci


### Importación de comandos de la libreria unthermal y de Matplotlib 

A continuación importamos los comandos básicos que usaremos de la libreria unmotor y la libreria Matplotlib para realizar trabajo adicional con las figuras. 

In [ ]:
%matplotlib widget
import matplotlib.pyplot as plt
from unmotor import ThermalSystemIoT, get_static_model, get_models_prbs, get_fomodel_step, step_closed


---


## 2. Definición del sistema en el IoT

Lo primero que necesitamos es definir el sistema y su conexión al IoT. Esto lo hacemos mediante el  comando

`ThermalSystemIoT(plant_number="xxxx", broker_address = "xx.xxx.xxx.xxx")`.

+ El parámetro `plant_number` es el identificador de la planta con que está trabajando. Por ejemplo `plant_number="PC-T-01"`

+ El parámetro `broker_address` es la dirección IP del broker al cual se conecta la planta. Por ejemplo ` broker_address = "10.203.162.69"` 

In [ ]:
my_thermal = ThermalSystemIoT(plant_number = 1010, broker_address = "10.203.162.69") 


## 3. Modelo estático del UNThermal

Es natural comenzar investigando el comportamiento estacionario de un sistema bajo
control. Esto se hace mediante una curva que muestra el valor en estado estacionario de la
salida (en este caso, la temperatura del par transistor-sensor) para diferentes valores de la variable de entrada (en este caso la potencia aplicada). A esta curva se le denomina Modelo Estático del Sistema.


El modelo estático se puede obtener experimentalmente de varias formas. *Una manera
es que el sistema esté en lazo cerrado con un controlador e ir ajustando diferentes valores de la referencia. Cuando la salida (en este caso la temperatura) alcanza el valor estacinario de referencia, se registra la señal de control que fue necesaria. Esto da un punto del modelo estático. Este procedimiento se repite para cubrir el rango completo de entradas.*

+ La  función `get_static_model(system)` nos permite obtener el modelo estático del UNThermal definido por la instancia del UNThermal creada con la función  `ThermalSystemIoT(.)`
+ La funcion `plt.savefig(.)` permite guardar la figura en en cualquier formato para su video informe

In [ ]:
ue, ye = get_static_model(my_thermal) 
plt.savefig("static_model.png", format="png", bbox_inches="tight")  



*Note que la salida los valores de estado estacionario de la entrada y la salida quedan registrados en las variables `ue`, `ye`. También quedan guardados en el archivo `\experiment_files\DCmotor_static_gain_response.csv` presente en la carpeta en la que ejecuta este script.*



### Preguntas sobre el modelo estático

Por favor responda las siguientes preguntas sobre el modelo estático en su videoinforme y haga una explicación preliminar muy breve en el documento compartido de la clase de laboratorio. Compare con lo que pasa con el motor DC. 

1.  Considere que el rango de operación va desde  $40^oC$ hasta $100^oC$. ¿Existen temperaturas en este rango más dificiles de controlar, según el modelo estático?
2.  Suponga que un modelo aproximado del sistema térmico está dado por $G(s)=\frac{\alpha}{\tau\,s+1}$. ¿Cual de los parámetros ($\alpha$ o $\tau$) podriamos asumir como constante, según la curva del modelo estático?
3.  ¿Cual es la máxima señal de control que tendriamos disponible para controlar una temperatura cercana a $60^oC$.

## 4. Modelo dinámico del UNThermal

A continuación vamos a obtener el modelo dinámico del UNDCMotor usando dos tipos de experimentos:

1. Modelo por respuesta al escalón
2. Modelo por respuesta a una secuencia binaria pseudoaleatoria.

---

### Modelo por Respuesta al escalón 

Cuando se realiza la linealización del modelo no lineal del UNThermal se obtiene un  [modelo lineal de primer orden](https://github.com/nebisman/UNDCMotor/blob/main/docs/modelo_matematico_unmotor.pdf).

Así, el UNThermal puede ser descrito por la siguiente función de transferencia en una región cercana al punto de operación:

$$G(s)= \frac{\alpha}{\tau\, s + 1}$$

Es usual que la primera prueba de identificación de sistemas que parecen de primer orden sea por la respuesta al escalón. 

La función `G = get_fomodel_step(system, yop = desired_op)`  *(get first order model by using step response)* permite obtener la respuesta al escalón y el modelo matemático del UNThermal por medio del [método de los 4 puntos](https://github.com/nebisman/UNDCMotor/blob/main/docs/metodo_4_puntos.pdf).

+ El parámetro `system` es la instancia del UNDCMotor creada con la función  `ThermalSystemIoT(.)`

+ El parámetro `yop = desired_op` permite obtener la respuesta al escalón con un cambio de potencia que inicia exactamente en el punto de operación `yop`.

+  La salida `G` retorna la función de transferencia del sistema.

En el ejemplo siguiente obtenemos la respuesta al escalón y el modelo para el punto de operación $y_{OP}=50 ^o/s$.



In [ ]:
G = get_fomodel_step(my_thermal, yop = 50)
plt.savefig("first_order_model_yop_100.png", format="png", bbox_inches="tight")


#### Experimento con el modelo por respuesta al escalón

Encuentre los modelos dinámicos por respuesta al escalón para los siguientes puntos de operación:

+ $y_{OP}=60^o\,C$
+ $y_{OP}=80^o\,C$



#### Preguntas sobre el modelo por respuesta al escalón

+ Describa como cambia la constante de tiempo $\tau$ al incrementar la temperatura
+ Describa como cambia la constante de ganancia $\alpha$ al incrementar la temperatura


---
  


### Modelo por respuesta a una señal binaria pseudoaleatoria (PRBS) 


Una forma de experimento más refinado para obtener el modelo lineal de un sistema en torno de un punto de operación es mediante el uso de **secuencias binarias pseudoaleatorias**. Este tipo de secuencias tienen una mayor riqueza espectral que los simples escalones y, por consiguiente, permiten obtener modelos lineales más refinados de un sistema en torno de un punto de operación.

El UNThermal puede ser descrito por  [la siguiente función de transferencia de primer orden](https://github.com/nebisman/UNDCMotor/blob/main/docs/modelo_matematico_unmotor.pdf):

$$G_1(s)= \frac{\alpha}{\tau\, s + 1} \text{ (Ec. 1)}.$$ 

Y, para aproximar el retardo introducido por el tiempo de muestreo se puede usar una función de transferencia de segundo orden, dada por: 

$$G_2(s)= \frac{\alpha}{\tau\, s + 1} \, e^{-\tau_D\,s} \text{ (Ec. 2)}.$$


La función `G1, G2 = get_models_prbs(system, yop = desired_speed)` (*get models by using a PRBS input*) permite obtener la respuesta a una entrada de tipo PRBS y encuentra los modelos de primer (FO) y de primer orden con retardo (FOTD) que mejor ajustan los datos.

+ El parámetro `system` es la instancia del UNDCMotor creada con la función  `ThermalSystemIoT(.)`

+ El parámetro `yop = desired_op` permite obtener el experimento con la entrada PRBS centrada en el punto de operación `yop`

+  Las salidas `G1` y `G2` retornan, respectivamente, las funciones de transferencia de primero y segundo orden del UNThermal.

En el ejemplo siguiente obtenemos la respuesta a una entrada PRBS para el punto de operación $y_{OP}=50 ^o/s$.



In [ ]:
G1, G2 = get_models_prbs(my_dcmotor, yop=50)
plt.savefig("PRBS_models_yop_100.png", format="png", bbox_inches="tight")  


#### Experimentos con la entrada PRBS

Encuentre los modelos dinámicos por por medio de la entrada PRBS para los siguientes puntos de operación:

+ $y_{OP}=60^o\, C$
+ $y_{OP}=80^o\, C$


#### Preguntas sobre los modelos con la entrada PRBS

+ Compare los modelos de primer orden con los obtenidos por la respuesta al escalón. ¿Se parecen?
+ Como difieren entre sí los modelos tomados a $60^o$ y a  $80^o$. Compare cualitativamente con las diferencias encontradas entre los diferentes puntos de operación en el UNMotor.
+ Según estos modelos, ¿Cúal de los dos sistemas (motor o sistema térmico) es más fácil de controlar?
  